# Constant Trace Creation

In this notebook we will produce a simulation of a constant flux through the science, LFC, sky background, and dark fibers, then combine them into a single image.

This is intended for measuring the trace/order positions and separations and is not intended as a realistic product.

In [ ]:
from pyechelle.simulator import Simulator
from pyechelle.telescope import Telescope
from pyechelle.sources import Constant
from pyechelle.spectrograph import ZEMAX
import numpy as np
from astropy.io import fits

### Setup paths and file names
Change the output path as desired.

In [ ]:
output_path = '/home/aidan/data/hispec_simulations/'

sci_fiber_model = 'HISPEC_HK_SCI'
lfc_fiber_model = 'HISPEC_HK_LFC'
sky_fiber_model = 'HISPEC_HK_SKY_BKGD'
dark_fiber_model = 'HISPEC_HK_DARK'

### Loop over each fiber
Each fiber will have the same constant flux.

<font color='red'>Note that this make take some time depending on the constant intensity!</font> You should see progress updates printed as each order is simulated.

In [ ]:
fibers = [sci_fiber_model, lfc_fiber_model, sky_fiber_model, dark_fiber_model]

for fiber_model in fibers:
    sim = Simulator(ZEMAX(fiber_model))
    sim.set_ccd(1) # always leave as 1
    sim.set_fibers(1) # always leave as 1
    sim.set_sources(Constant(1e-5)) # Constant(intensity [microW / micron / s])
    sim.set_exposure_time(1) # s
    sim.set_output(output_path+fiber_model+'_constant_trace.fits', overwrite=True)
    sim.set_telescope(Telescope(9.96,0)) # Telescope(primary diameter, secondary diameter)
    sim.set_orders([*range(59, 98)]) # a list of all expected orders 59 to 97

    sim.max_cpu = 1 # set number of cpu to use
    sim.run()

### Combine into a single image

In [ ]:
hdu_data = np.empty((4, 4096, 4096))

for i, fiber_model in enumerate(fibers):
    hdu = fits.open(output_path+fiber_model+'_constant_trace.fits')
    hdu_data[i] = hdu[0].data
    hdu.close()
    
data = np.sum(hdul, axis=0)

# Write result
dataout = fits.PrimaryHDU(data)
dataout.writeto(output_path+'HISPEC_HK_constant_trace_combined.fits', overwrite=True)